In [108]:
import pandas as pd
import numpy as np
import psycopg2
from scipy import stats

In [21]:
conn = psycopg2.connect('dbname=football_db')

In [114]:
cur = conn.cursor()

In [122]:
columns = ['id', 'home_goal', 'away_goal', 'result']

query = f"""
SELECT match_api_id, home_team_goal, away_team_goal, 
CASE WHEN home_team_goal > away_team_goal THEN 'Win' 
         ELSE 'NoWin' END as Result
FROM Match
"""

cur.execute(query)
data = cur.fetchall()

In [123]:
df = pd.DataFrame(data, columns=columns)

In [124]:
df.head()

,id,home_goal,away_goal,result
0,492473,1,1,NoWin
1,492474,0,0,NoWin
2,492475,0,3,NoWin
3,492476,5,0,Win
4,492477,1,3,NoWin


In [125]:
total_wins = len(df[df['result'] == 'Win'])
total_wins

11917

In [126]:
total_games = len(df['result'])
total_games

25979

In [120]:
mu = total_wins / total_games

In [121]:
mu

0.28738596558759

In [102]:
samples = np.zeros(100)
for i in range(100):
    games = df.iloc[np.random.randint(low=0, high=len(df), size=100), :]
    win_rate = len(games[games['result'] == 'Win']) / 100
    samples[i] = win_rate


In [103]:
sample_mean = samples.mean()
sample_mean

0.4581

In [104]:
std = np.std(samples, ddof=1)
std

0.052851899041786866

In [105]:
t = (sample_mean - mu) / (std / np.sqrt(100))
t

-0.11667617845374012

In [106]:
df = 99

In [111]:
t_crit = np.round(stats.t.ppf(1 - 0.05, df), 3)
t_crit

1.66

In [112]:
results = stats.ttest_1samp(a=samples, popmean=mu)

In [113]:
results

Ttest_1sampResult(statistic=-0.11667617845374012, pvalue=0.9073530328525758)

In [23]:
columns = ['team_short_name', 'team_api_id', 'Category', 'Result', 'Count' ]

query = f"""
SELECT A.team_short_name as Team, B.* FROM
(SELECT T.team_api_id, 'Home' as Category,
CASE WHEN M1.home_team_goal - M1.away_team_goal > 0 THEN 'Win' ELSE 'No' END as Result,
COUNT(*) as Count
FROM Team T
JOIN Match M1 ON T.team_api_id = M1.home_team_api_id
GROUP BY T.team_api_id, Result
UNION SELECT T.team_api_id, 'Away' as Category,
CASE WHEN M1.home_team_goal - M1.away_team_goal < 0 THEN 'Win' ELSE 'No' END as Result,
COUNT(*) as Count
FROM Team T
JOIN Match M1 ON T.team_api_id = M1.away_team_api_id
GROUP BY T.team_api_id, Result) as B
JOIN Team A ON B.team_api_id = A.team_api_id
"""

cur.execute(query)
data = cur.fetchall()

In [24]:
df = pd.DataFrame(data, columns=columns)

In [25]:
df.head(5)

,team_short_name,team_api_id,Category,Result,Count
0,CHO,1601,Away,No,82
1,CHO,1601,Away,Win,38
2,CHO,1601,Home,No,66
3,CHO,1601,Home,Win,54
4,O-H,1773,Away,No,39


In [26]:
columns = ['team_short_name', 'team_api_id']

query = f"""
SELECT {', '.join(columns)}
FROM Team;
"""

In [27]:
cur.execute(query)
data = cur.fetchall()

In [28]:
team_df = pd.DataFrame(data, columns=columns)
team_df.head()

,team_short_name,team_api_id
0,GEN,9987
1,BAC,9993
2,ZUL,10000
3,LOK,9994
4,CEB,9984


In [ ]:
np.random.seed(0)
samples = df.iloc[np.random.randint(low=0, high=len(df), size=100), :]

In [ ]:
mu = df[df['Home_Result'] == "Win"].count() / df.count()
mu

In [ ]:
x_bar = samples[samples['Home_Result'] == "Win"].count() / samples.count()
x_bar

In [ ]:
columns = ['Home_Result', 'Count']

query = f"""
SELECT Home_Result, COUNT(*) as Count FROM
(SELECT B.team_short_name as Home,  C.team_short_name as Away,  
(CASE WHEN M.home_team_goal - M.away_team_goal > 0 THEN 'Win' 
      WHEN  M.home_team_goal - M.away_team_goal = 0 THEN 'Draw'
      ELSE 'Lose' END) as Home_Result 
FROM Match M
JOIN Team B ON M.home_team_api_id = B.team_api_id
JOIN Team C ON M.away_team_api_id = C.team_api_id) AS T
GROUP BY Home_Result
"""

cur.execute(query)